## The Challenge

We have a research analyst team who provide recommendations to our portfolio managers. Unfortunately not all of these recommendations lead to a positive outcome. We need a better way to stress test the recommendations of our analyst team in a live environment to ensure that they are robust before they are passed onto our portfolio managers.

We also need to understand what information an analyst was presented with when they made a recommendation. This will allow us to understand the reasoning behind the decision at the time that it was made. This has been very difficult for us to determine previously.

## The Solution

Using LUSID we can construct a "paper portfolio" for each member of our analyst team. Using this paper portfolio they will be able to make trades based on their own recommendations to see how they would have performed under close to real market conditions. We can do this by:

1) Creating a logically separate environment for each analyst

2) Creating our tradeable instrument universe for each analyst

3) Creating a transaction portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolio for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

4) Populating each analyst's portfolio with a starting cash balance

5) Populating each analyst's reference portfolio with the weights for each constituent of their index

6) Allowing our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

7) Creating an analytics store to hold the latest prices of our tradeable instrument universe

8) Valuing our analyst's portfolio using our analytics store

9) Creating an index from our reference portfolio and building a benchmark for our analyst's to compare their performance against

*To initialise our LUSID environment run the cell below*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

LUSID Environment Initialised


![Scopes](img/paper-lusid.gif)

## 1) Creating a logically separated environment for each analyst

In LUSID we can create separate environments for each of our analysts using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that an analyst can only interact with objects inside their scope. Furthermore no other analyst or other stakeholder inside the organisation can access this scope unless explicitly given permission. 

For this example we will create a scope for a single analyst. 

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [2]:
# Fetch our scopes
scope_id = import_data.create_scope_ids(1)
analyst_scope_code = 'analyst-paper-{}'.format(scope_id[0])

prettyprint.heading('Analyst Scope Code', analyst_scope_code)

Analyst Scope Code: analyst-paper-ec4a


*Our LUSID environment now looks like the below. We will update this with each step.*

![Scopes](img/paper-scopes.gif)

## 2) Creating our tradeable instrument universe for each analyst

Before we can set-up our transaction and reference portfolios we need to define the tradeable instrument universe for our analyst. In this case let us assume that the analyst has been tasked with making recommendations to buy or sell equities from the London Stock Exchange FTSE techMark Focus index.

*Run the cell below to import our instrument universe*

In [3]:
instrument_universe = import_data.fetch_instrument_universe(
    'paper-instruments.csv', 
    paper=True)

prettyprint.instruments(instrument_universe, 'Figi')

AVEVA GRP: Figi - BBG000C21Y87
BAE SYS.: Figi - BBG000BD5TW4
BATM ADVANCED: Figi - BBG000BFJD77
BTG: Figi - BBG000BZZK79
COBHAM: Figi - BBG000BS6810
COMPUTACENTER: Figi - BBG000BN7CL9
CONSORT MED.: Figi - BBG000BD8278
DIALIGHT: Figi - BBG000BLQNG4
GENUS: Figi - BBG000CTMKX7
GRESHAM TECH: Figi - BBG000BS2592
KCOM GRP: Figi - BBG000BYWT85
MEGGITT: Figi - BBG000BDT1M0
MICRO FOCUS: Figi - BBG000G4KJC6
MICROGEN: Figi - BBG000BDSS88
NCC GRP: Figi - BBG000BS8WF0
OXFORD BIOMED.: Figi - BBG000BMB7T9
OXFORD INSTRMNT: Figi - BBG000BDWZZ7
PADDY PWR BET: Figi - BBG000N8GHT0
QINETIQ: Figi - BBG000HF3Y54
RENISHAW: Figi - BBG000BDZTQ8
RICARDO: Figi - BBG000BDYQS3
SAGE GRP.: Figi - BBG000BN0PP3
SDL: Figi - BBG000C4HGK1
SMITH&NEPHEW: Figi - BBG000BF21F2
SPECTRIS: Figi - BBG000CNV8Z5
SPIRENT: Figi - BBG000C4LV50
TRIAD GRP.: Figi - BBG000BL9R74
ULTRA ELEC.: Figi - BBG000HHH4C3
VECTURA: Figi - BBG000PRWXX8
XAAR: Figi - BBG000BVKQF2
XP POWER: Figi - BBG000BZMBQ2


Now that we have our instruments, we can upsert our instruments in a batch. You can read more about upserting instruments in the LUSID API documentation - [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instruments into LUSID*

In [4]:
# Initialise our batch upsert request
batch_upsert_request = {}
# Using our instrument universe create our batch request
for instrument_name, instrument in instrument_universe.items():
    # Add the ISIN to our instrument as a property
    isin_property = models.InstrumentProperty(
        "Instrument/default/Isin", 
        models.PropertyValue(instrument['identifiers']['Isin']))
    # Add the Sedol to our instrument as a property
    sedol_property = models.InstrumentProperty(
        "Instrument/default/Sedol", 
        models.PropertyValue(instrument['identifiers']['Sedol']))
    # Add the instrument to our batch request using the FIGI as the main unique identifier
    batch_upsert_request[instrument_name] = models.InstrumentDefinition(
        name=instrument_name,
        identifiers={'Figi': instrument['identifiers']['Figi']},
        properties=[isin_property, sedol_property])
                                                                           
# Call LUSID to upsert our batch
instrument_response = client.upsert_instruments(requests=batch_upsert_request)

# Pretty print the response from LUSID
prettyprint.instrument_response(instrument_response, identifier='Figi')

Instrument Successfully Upserted: TRIAD GRP.
Figi ID: BBG000BL9R74
LUSID Instrument ID: LUID_LR5ET8PZ


Instrument Successfully Upserted: COBHAM
Figi ID: BBG000BS6810
LUSID Instrument ID: LUID_2X18R9IK


Instrument Successfully Upserted: SDL
Figi ID: BBG000C4HGK1
LUSID Instrument ID: LUID_2ECEHQEH


Instrument Successfully Upserted: SMITH&NEPHEW
Figi ID: BBG000BF21F2
LUSID Instrument ID: LUID_ZM1SI6QC


Instrument Successfully Upserted: VECTURA
Figi ID: BBG000PRWXX8
LUSID Instrument ID: LUID_DU44SE3A


Instrument Successfully Upserted: BAE SYS.
Figi ID: BBG000BD5TW4
LUSID Instrument ID: LUID_99SV40PB


Instrument Successfully Upserted: OXFORD BIOMED.
Figi ID: BBG000BMB7T9
LUSID Instrument ID: LUID_IVR0TVNL


Instrument Successfully Upserted: NCC GRP
Figi ID: BBG000BS8WF0
LUSID Instrument ID: LUID_JKS4UYOQ


Instrument Successfully Upserted: MICRO FOCUS
Figi ID: BBG000G4KJC6
LUSID Instrument ID: LUID_LU19QSZV


Instrument Successfully Upserted: SPECTRIS
Figi ID: BBG000CNV8Z5
LUSID Instr

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/paper-instrumentmaster.gif)

During the upsertion process you may have noticed that that every instrument that is created in LUSID given a unique LUSID Instrument Id or LUID for short. This ID is used for many methods and is how LUSID uniquely identifies an instrument.

We therefore want to add our newly created LUIDs to our instrument list so that we can easily reference our instruments in future.

Note that in addition to our instruments, there is also cash in our holdings. Instruments that represent cash are already set-up in LUSID by default. In LUSID the LUID for cash is the prefix CCY_ followed by the ISO4217 currency code. For example British Pounds have a LUID of CCY_GBP. We will add these to our local instrument universe so that we can reference them.

*Run the cell below to add our LUIDs and cash to our local instrument universe*

In [5]:
# Loop over our recently upserted instruments
for instrument_name, instrument in instrument_response.values.items():
    # Add our LUID as a new identifier so that we can use it in our calls later
    instrument_universe[instrument_name]['identifiers']['LUID'] = instrument.lusid_instrument_id
    
# We will also add some cash instruments to our universe
instrument_universe['GBP_Cash'] = {
    'identifiers': {'LUID': 'CCY_GBP'},
    'currency': 'GBP'}

# Pretty print our updated local instrument universe
prettyprint.instruments(instrument_universe, 'LUID')

AVEVA GRP: LUID - LUID_C2G6UXB1
BAE SYS.: LUID - LUID_99SV40PB
BATM ADVANCED: LUID - LUID_0L5AS1JO
BTG: LUID - LUID_EKJN8O19
COBHAM: LUID - LUID_2X18R9IK
COMPUTACENTER: LUID - LUID_VB0W6R8R
CONSORT MED.: LUID - LUID_6QBS2E6F
DIALIGHT: LUID - LUID_8W2UDUJ0
GENUS: LUID - LUID_NV3D9039
GRESHAM TECH: LUID - LUID_Y4R8RLXK
KCOM GRP: LUID - LUID_V8E59UIX
MEGGITT: LUID - LUID_IS8O0NPO
MICRO FOCUS: LUID - LUID_LU19QSZV
MICROGEN: LUID - LUID_42QU0Z23
NCC GRP: LUID - LUID_JKS4UYOQ
OXFORD BIOMED.: LUID - LUID_IVR0TVNL
OXFORD INSTRMNT: LUID - LUID_6AO46VL8
PADDY PWR BET: LUID - LUID_L5J8JP01
QINETIQ: LUID - LUID_WS2AQ3EJ
RENISHAW: LUID - LUID_G5OMBNL4
RICARDO: LUID - LUID_9W5VPPUW
SAGE GRP.: LUID - LUID_K499YGKV
SDL: LUID - LUID_2ECEHQEH
SMITH&NEPHEW: LUID - LUID_ZM1SI6QC
SPECTRIS: LUID - LUID_KYXYUGO7
SPIRENT: LUID - LUID_FMEJVLHI
TRIAD GRP.: LUID - LUID_LR5ET8PZ
ULTRA ELEC.: LUID - LUID_VO28IAUC
VECTURA: LUID - LUID_DU44SE3A
XAAR: LUID - LUID_KA3S9XLZ
XP POWER: LUID - LUID_7AJLUKPO
GBP_Cash: LUID

## 3) Creating a transactions portfolio for each analyst to hold their tradeable instrument universe and creating a reference portfolo for each analyst which contains the index of their tradeable instrument universe and can be used as a benchmark for performance

Now that we have populated our tradeable instrument universe, we need to create our analyst's transaction portfolio for making trades and their reference portfolio for tracking the index. Each portfolio can be referenced by its unique 'code'. Let us define the code for our two portfolios.

*Run the cell below to define our portfolio codes*

In [6]:
# Define unique codes for each of our portfolios
transaction_portfolio_code = 'LSE_FTSE_techMarkFocus_transactions'
reference_portfolio_code = 'LSE_FTSE_techMarkFocus_reference'
# Pretty print our codes
prettyprint.heading('Transaction Portfolio Code', transaction_portfolio_code)
prettyprint.heading('Reference Portfolio Code', reference_portfolio_code)

Transaction Portfolio Code: LSE_FTSE_techMarkFocus_transactions
Reference Portfolio Code: LSE_FTSE_techMarkFocus_reference


Now that we have the code for our portfolios we can go ahead and create them in LUSID. Let us first create our transaction portfolio. Note that we will create both portfolios as being available as of 3 days ago. This will allow us to add backdated transactions and compute backdated valuations.

You can read more about creating transaction portfolios in the LUSID API documentation: [LUSID API Docs: Creating Transaction Portfolios](https://docs.lusid.com/#operation/CreatePortfolio)

*Run the cell below to create our transaction portfolio*

In [7]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=transaction_portfolio_code,
    code=transaction_portfolio_code,
    base_currency='GBP',
    description='Paper transaction portfolio',
    created=portfolio_creation_date)

# Call LUSID to create our portfolio
portfolio_response = client.create_portfolio(
    scope=analyst_scope_code,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-ec4a
Code: LSE_FTSE_techMarkFocus_transactions
Portfolio Effective From: 2019-01-25 16:44:13.713512+00:00
Portfolio Created On: 2019-01-28 16:44:14.104959+00:00



![Scopes](img/paper-transactionsportfolio.gif)

Now that we have created our transaction portfolio we can also create our referene portfolio to track the techMarkFocus index. 

You can read more about creating reference portfolios in the LUSID API documentation: [LUSID API Docs: Creating Reference Portfolios](https://docs.lusid.com/#operation/CreateReferencePortfolio)

*Run the cell below to create our reference portfolio*

In [8]:
# The date our portfolios were first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=3))

# Create the request to add our portfolio
reference_portfolio_request = models.CreateReferencePortfolioRequest(
    display_name=reference_portfolio_code,
    code=reference_portfolio_code,
    description='Paper reference portfolio',
    created=portfolio_creation_date)

# Call LUSID to create our reference portfolio
portfolio_response = client.create_reference_portfolio(
    scope=analyst_scope_code,
    reference_portfolio=reference_portfolio_request)

# Pretty print our response from LUSID
prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-ec4a
Code: LSE_FTSE_techMarkFocus_reference
Portfolio Effective From: 2019-01-25 16:44:14.329771+00:00
Portfolio Created On: 2019-01-28 16:44:15.595961+00:00



Our LUSID environment is now populated with our two portfolios. One for our analyst to make trades across and the other to track the weights of the index of the tradeable instrument universe to produce a benchmark.

![Scopes](img/paper-referenceportfolio.gif)

## 4) Populating our analyst's portfolio with a starting cash balance

To set our starting cash balance we use LUSID's 'Set Holdings' capability. This allows us to set the holdings of a portfolio to a given state. In this case we will add in a starting cash balance. Let us give the analyst a starting cash balance of £30,000,000. We will make this balance effective from just after the creation of our portfolio. 

*Run the cell below to set our starting cash balance*

In [9]:
# Set the date from which the cash balance will apply to be just after portfolio creation
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=2.9))
# Define our initial cash balance
initial_cash_balance = 30000000

# Create a holding adjustment to set our initial cash balance
holding_adjustment = [
    models.AdjustHoldingRequest(
        instrument_uid='CCY_GBP',
        tax_lots=[
            models.TargetTaxLotRequest(
                units=initial_cash_balance,
                cost=models.CurrencyAndAmount(
                    amount=initial_cash_balance,
                    currency='GBP'),
                portfolio_cost=initial_cash_balance,
                price=1)
                ]
    )
]

# Call LUSID to set our initial cash balance
set_holdings_response = client.set_holdings(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustment)

# Pretty print our response from LUSID
prettyprint.set_holdings_response(
    set_holdings_response, 
    analyst_scope_code, 
    transaction_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: analyst-paper-ec4a
Code: LSE_FTSE_techMarkFocus_transactions
Holdings Effective From: 2019-01-25 19:08:15.799201+00:00
Holdings Created On: 2019-01-28 16:44:16.295804+00:00



Our transactions portfolio is now populated with an initial cash balance and is no longer empty.

![Scopes](img/paper-transactionsportfoliofilled.gif)

## 5) Populating our reference portfolio with the correct weights so that it tracks the index as closely as possible

To set up our reference portfolio we need to initialise it with the constituents that make up the index as well as their weight. In this case we are going to weight the consitutents based on their market capitalisation. We will import their market capitalisation from a CSV file generated from one of our internal systems. We will also configure our reference portfolio to be updated once a quarter. We could update this more frequently if we wished. 

*Run the cell below to import our market capitalisation figures for our instruments*

In [10]:
# Import the market capitalisation of each of our instruments in the index
instrument_market_cap = import_data.fetch_instrument_market_cap('paper-weights.csv')
# Pretty print the market caps
prettyprint.instruments_market_cap(instrument_market_cap)

AVEVA GRP: £4502.68 (millions)
BAE SYS.: £16349.66 (millions)
BATM ADVANCED: £185.25 (millions)
BTG: £3202.2 (millions)
COBHAM: £2579.9 (millions)
COMPUTACENTER: £1161.96 (millions)
CONSORT MED.: £422.56 (millions)
DIALIGHT: £133.39 (millions)
GENUS: £1455.72 (millions)
GRESHAM TECH: £64.34 (millions)
KCOM GRP: £359.04 (millions)
MEGGITT: £3937.11 (millions)
MICRO FOCUS: £6199.1 (millions)
MICROGEN: £238.57 (millions)
NCC GRP: £503.69 (millions)
OXFORD BIOMED.: £449.57 (millions)
OXFORD INSTRMNT: £496.87 (millions)
PADDY PWR BET: £4912.58 (millions)
QINETIQ: £1732.29 (millions)
RENISHAW: £3054.21 (millions)
RICARDO: £331.12 (millions)
SAGE GRP.: £6769.89 (millions)
SDL: £505.93 (millions)
SMITH&NEPHEW: £12646.09 (millions)
SPECTRIS: £2831.96 (millions)
SPIRENT: £867.45 (millions)
TRIAD GRP.: £5.46 (millions)
ULTRA ELEC.: £931.48 (millions)
VECTURA: £490.72 (millions)
XAAR: £118.58 (millions)
XP POWER: £402.01 (millions)


Now that we have our market capitalisation figures we can upsert our constituents into the reference portfolio. We can add the weights in any format and LUSID will normalise them to equal 1. 

You can read more about upserting constituents in the LUSID API documentation: [LUSID API Docs: Upserting Constituents to a Reference Portfolio](https://docs.lusid.com/#operation/UpsertReferencePortfolioConstituents)

*Run the cell below to upsert our constituents*

In [11]:
# Initialise a list to hold our constituents
constituents = []
# Work out the total market capitalisation of the entire index
total = sum(instrument_market_cap.values())

# Iterate over instrument unvierse to add each constituent to our list
for instrument_name, instrument in instrument_universe.items():
    # Skip cash instruments
    if 'Cash' in instrument_name:
        continue
    # Calculate our constituents weight based on market cap and add it to our list
    constituents.append(models.ReferencePortfolioConstituentRequest(
        instrument_uid=instrument['identifiers']['LUID'],
        weight=instrument_market_cap[instrument_name]/total,
        currency=instrument['currency']))

# Create our request to add our constituents
constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
    effective_from=holdings_effective_date,
    weight_type="Periodical",
    period_type="Quarterly",
    period_count=4,
    constituents=constituents)

# Call LUSID to upsert our constituents into our reference portfolio
response = client.upsert_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    constituents=constituents_request)

print ('Constituents Upserted')

Constituents Upserted


Our reference portfolio is no longer empty and now contains the weights for each instrument in our index against which the analyst's performance will be compared.

![Scopes](img/paper-referenceportfoliofilled.gif)

## 6) Allow our analysts to trade across their tradeable instrument universe and add their transactions to their transaction portfolio

Now that we have set up our portfolios we can add in our analyst's trades. These would likel be populated from an order management or similar system. In this case we are going to import the trades over the last two days from a CSV file.

*Run the cell below to import our transactions*

In [12]:
# Import our analyst's trades from a CSV file
analyst_transactions = import_data.fetch_client_transactions(
    'paper-transactions.csv', 
    instrument_universe,
    days_back=2)

# Pretty print their activity
prettyprint.transactions_strategy(analyst_transactions)

Transaction: tid_124398219481
Portfolio: LSE_FTSE_techMarkFocus_transactions
Instrument Name: AVEVA GRP
Instrument LUID: LUID_C2G6UXB1
Type: Buy
Units: 265600
Price: 28.94
Total Cost: 7686464.0
Currency: GBP
Transaction Date: 2019-01-27T11:35:15.394524+00:00
Settlement Date: 2019-01-29T11:35:15.394524+00:00
Strategy: quantitativeSignal

Transaction: tid_339423984894
Portfolio: LSE_FTSE_techMarkFocus_transactions
Instrument Name: GBP_Cash
Instrument LUID: CCY_GBP
Type: FundsOut
Units: 265600
Price: 0.005788000000000001
Total Cost: 1537.2928
Currency: GBP
Transaction Date: 2019-01-27T09:27:23.356699+00:00
Settlement Date: 2019-01-29T09:27:23.356699+00:00
Strategy: quantitativeSignal

Transaction: tid_339423984896
Portfolio: LSE_FTSE_techMarkFocus_transactions
Instrument Name: GBP_Cash
Instrument LUID: CCY_GBP
Type: FundsOut
Units: 265600
Price: 0.014469999999999998
Total Cost: 3843.232
Currency: GBP
Transaction Date: 2019-01-27T12:38:40.079779+00:00
Settlement Date: 2019-01-29T12:38:40.0

We can set custom properties on our transactions. In this case we are going to create a strategy property which allows the analyst to note what strategy they were following when they made the trade. These will help us justify their trades and recommendations when it comes to making them for real. 

You can read more about creating properties in the LUSID API documentation: [LUSID API Docs: Defining a new Property](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our transaction strategy property*

In [13]:
# Create a request to define our strategy property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Trade',
    scope=analyst_scope_code,
    code='strategy',
    value_required=False,
    display_name='strategy',
    data_type_id=models.ResourceId(
        scope='default',
        code='string')
)

# Call LUSID to create our new property
property_response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our strategy property key
prettyprint.heading('Strategy Property Key: ', strategy_property_key)

Strategy Property Key: : Trade/analyst-paper-ec4a/strategy


Now that we have our trades imported and our property created we can upsert them into LUSID. Read more about adding transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions into our transactions portfolio*

In [14]:
# Initialise a list to hold our transactions
batch_transaction_requests = []

# Iterate over the transactions for each portfolio
for transaction_id, transaction in analyst_transactions.items():
    
    batch_transaction_requests.append(
        models.TransactionRequest(
            transaction_id=transaction_id,
            type=transaction['type'],
            instrument_uid=transaction['instrument_uid'],
            transaction_date=transaction['transaction_date'],
            settlement_date=transaction['settlement_date'],
            units=transaction['units'],
            transaction_price=models.TransactionPrice(
                price=transaction['transaction_price'],
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=transaction['total_cost'],
                currency=transaction['transaction_currency']),
            source='Client',
            transaction_currency=transaction['transaction_currency'],
            properties={
                strategy_property_key: models.PropertyValue(
                    label_value=transaction['strategy'])}
             ))

# Call LUSID to upsert our transactions
transaction_response = client.upsert_transactions(
    scope=analyst_scope_code,
    code=transaction_portfolio_code,
    transactions=batch_transaction_requests)

# Pretty print the response from LUSID
prettyprint.transactions_response(
    transaction_response,
    analyst_scope_code, 
    transaction_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: analyst-paper-ec4a
Code: LSE_FTSE_techMarkFocus_transactions
Transactions Effective From: 2019-01-27 16:48:48.573874+00:00
Transactions Created On: 2019-01-28 16:44:18.112951+00:00



Our transaction portfolio is now populated with our analyst's paper trades.

## 7) Create an analytics store to hold the latest prices of our tradeable instrument universe

To value our portfolios we need current prices. We store these prices in an analytics store. There are a number of different ways and sources that we can get our pricing information. In this case we are going to load the prices when we set up our portfolios and the current prices from a CSV file.

*Run the cell below to import our pricing information*

In [15]:
# Import our instrument prices from a CSV file
instrument_prices = import_data.fetch_instrument_analytics('paper-analytics.csv')
# Pretty print our pricing
prettyprint.instrument_prices(instrument_prices)

Instrument :AVEVA GRP
Price 3 Days Ago :£27.84
Price Today :£30.32

Instrument :BAE SYS.
Price 3 Days Ago :£5.15
Price Today :£4.78

Instrument :BATM ADVANCED
Price 3 Days Ago :£0.46
Price Today :£0.42

Instrument :BTG
Price 3 Days Ago :£8.28
Price Today :£8.97

Instrument :COBHAM
Price 3 Days Ago :£1.07
Price Today :£1.02

Instrument :COMPUTACENTER
Price 3 Days Ago :£10.68
Price Today :£11.46

Instrument :CONSORT MED.
Price 3 Days Ago :£8.7
Price Today :£9.31

Instrument :DIALIGHT
Price 3 Days Ago :£3.92
Price Today :£3.55

Instrument :GENUS
Price 3 Days Ago :£22.46
Price Today :£20.6

Instrument :GRESHAM TECH
Price 3 Days Ago :£0.94
Price Today :£1.0

Instrument :KCOM GRP
Price 3 Days Ago :£0.7
Price Today :£0.73

Instrument :MEGGITT
Price 3 Days Ago :£5.04
Price Today :£4.8

Instrument :MICRO FOCUS
Price 3 Days Ago :£14.79
Price Today :£13.82

Instrument :MICROGEN
Price 3 Days Ago :£3.87
Price Today :£3.6

Instrument :NCC GRP
Price 3 Days Ago :£1.81
Price Today :£1.81

Instrument :O

Now that we have our pricing information we can store this data inside LUSID in an analytics store. We will create two stores one for the prices from 3 days ago when we originally set-up the portfolios and one for the current prices.

You can read more abouy creating and setting analytics stores in the LUSID API documentation: [LUSID API Docs: Creating an Analytics Store](https://docs.lusid.com/#operation/CreateAnalyticStore) & [LUSID API Docs: Setting Analytic Data](https://docs.lusid.com/#operation/SetAnalytics)

*Run the cell below to set-up our analytics stores and set our analytic data*

In [16]:
# Set our analytics effective dates
analytics_effective_date = datetime.now(pytz.UTC) - timedelta(days=3)
today = datetime.now(pytz.UTC)
analytics_store_dates = [analytics_effective_date, today]

# Create prices via instrument, analytic
instrument_analytics_setup = []
instrument_analytics_today = []

# Create dummy prices of 1 for all instruments except cash
for instrument_name, instrument_analytics in instrument_prices.items():
    if 'Cash' in instrument_name:
        continue
        
    instrument_analytics_setup.append(models.InstrumentAnalytic(
        instrument_uid=instrument_universe[instrument_name]['identifiers']['LUID'],
        value=instrument_analytics[0]))
    
    instrument_analytics_today.append(models.InstrumentAnalytic(
        instrument_uid=instrument_universe[instrument_name]['identifiers']['LUID'],
        value=instrument_analytics[1]))
    
instrument_analytics = [instrument_analytics_setup, instrument_analytics_today]
    
for date, instrument_analytic_set in zip(analytics_store_dates, instrument_analytics):
    
    # Create analytics store request
    analytics_store_request = models.CreateAnalyticStoreRequest(scope=analyst_scope_code,
                                                                date_property=date.isoformat())

    # Call LUSID to create our analytics store
    client.create_analytic_store(request=analytics_store_request)

    # Call LUSID to set up our newly created analytics store with our prices
    client.set_analytics(scope=analyst_scope_code,
                         year=date.year,
                         month=date.month,
                         day=date.day,
                         data=instrument_analytic_set)
    
print ('Analytics Set')

Analytics Set


Our LUSID environment now looks like the below. We have a reference portfolio to hold our index which will act as a benchmark, we have a transaction portfolio for our analyst to make paper trades and we have an analytics store to value our portfolios.

![Scopes](img/paper-analyticsstore.gif)

## 8) Value each portfolio using our analytics store

We can now value our portfolios to see if the analyst has been making good decisions. Let us start by valuing our transaction portfolio to find out what our absolute return is across our tradeable instrument universe. 

We do this by making an aggregation request to LUSID. We will aggregate by instrument against the number of units, cost, present value and price. 

You can read more about aggregation in the LUSID API documentation: [LUSID API Docs: Aggregate Data in a Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio)

*Run the cell below to make our aggregation request on our transaction portfolio and see how our analyst is performing*

In [17]:
# Create our aggregation request
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=analyst_scope_code,
        code='default'),
        effective_at=today,
        metrics=[
            models.AggregateSpec(
                key='Holding/default/SubHoldingKey',
                op='value'),
            models.AggregateSpec(
                key='Holding/default/Units',
                op='sum'),
            models.AggregateSpec(
                key='Holding/default/Cost',
                op='sum'),
            models.AggregateSpec(
                key='Holding/default/PV',
                op='sum'),
           models.AggregateSpec(
               key='Holding/default/Price',
               op='sum') 
        ])
                                                        
# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.get_aggregation_by_portfolio(scope=analyst_scope_code,
                                                                code=transaction_portfolio_code,
                                                                request=aggregation_request)

prettyprint.aggregation_response_paper(aggregated_portfolio)

Instrument :LusidInstrumentId=LUID_VO28IAUC/GBP
Units :120000.0
Current Price :£13.36
Present Value :£1603546.8
Cost :£1582800.0
Return :1.3108%

Instrument :LusidInstrumentId=LUID_0L5AS1JO/GBP
Units :750000.0
Current Price :£0.42
Present Value :£318431.25
Cost :£347250.0
Return :-8.2991%

Instrument :LusidInstrumentId=LUID_2ECEHQEH/GBP
Units :55000.0
Current Price :£5.6
Present Value :£308154.0
Cost :£305250.0
Return :0.9514%

Instrument :LusidInstrumentId=LUID_K499YGKV/GBP
Units :-98000.0
Current Price :£6.57
Present Value :£-643432.72
Cost :£-577220.0
Return :-11.471%

Instrument :LusidInstrumentId=LUID_VB0W6R8R/GBP
Units :-150000.0
Current Price :£11.46
Present Value :£-1718946.0
Cost :£-1575000.0
Return :-9.1394%

Instrument :LusidInstrumentId=LUID_C2G6UXB1/GBP
Units :265600.0
Current Price :£30.32
Present Value :£8052397.06
Cost :£7686464.0
Return :4.7607%

Instrument :LusidInstrumentId=LUID_8W2UDUJ0/GBP
Units :1500000.0
Current Price :£3.55
Present Value :£5327280.0
Cost :£62700

## 9) Compare performance across the analyst's reference and transaction portfolios

Now that we have got our analyst's absolute return we want to see how this compares to their benchmark. In LUSID we can't value our reference portfolio directly as all it contains is weights. What we can is decide on an arbitary index level (in this case we will start with 1000), create a new transaction portfolio to hold our index and then populate it based on the weights from our reference portfolio.

First things first, let us give our index portfolio a code.

*Run the cell below to create a unique code for our index portfolio*

In [18]:
index_portfolio_code = 'LSE_FTSE_techMarkFocus_index'
prettyprint.heading('Index Portfolio Code', index_portfolio_code)

Index Portfolio Code: LSE_FTSE_techMarkFocus_index


Now we can create our index portfolio

*Run the cell below to create our index portfolio*

In [19]:
# Create the request to add our portfolio
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=index_portfolio_code,
    code=index_portfolio_code,
    base_currency='GBP',
    description='Index transaction portfolio',
    created=portfolio_creation_date)

# Create our portfolio
portfolio_response = client.create_portfolio(
    scope=analyst_scope_code,
    create_request=transaction_portfolio_request)

prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: analyst-paper-ec4a
Code: LSE_FTSE_techMarkFocus_index
Portfolio Effective From: 2019-01-25 16:44:14.329771+00:00
Portfolio Created On: 2019-01-28 16:44:22.257219+00:00



We now have an index portfolio that we can use to hold our index and obtain our benchmark.

![Scopes](img/paper-indexportfolio.gif)

In [20]:
# Set an arbitary index level to start our index with
index_level = 1000
# Call LUSID - get the constituents of our index from our reference portfolio
constituents = client.get_reference_portfolio_constituents(
    scope=analyst_scope_code,
    code=reference_portfolio_code,
    effective_at=datetime.now(pytz.UTC))
# Initialise our list to hold the adjustments we need to make to our index to set it up
index_setup = []
# Get our weights from the constituents into a better format to work with
weights = {constituent.instrument_uid:constituent.weight for constituent in constituents.constituents}
# Iterate over our pricing analytics
for instrument_name, instrument_analytics in instrument_prices.items():
    # Get our Lusid Unique Identifier
    Luid = instrument_universe[instrument_name]['identifiers']['LUID']
    # Get the initial price for each constituent of the index from our analytics store
    index_price = instrument_analytics[0]
    # Work out how much of the index this constituent should make up using its w
    index_cost = weights[Luid] * index_level
    # Work out how many units we should therefore buy
    index_units = index_cost / index_price
    # Create our request for this instrument 
    index_setup.append(
        models.AdjustHoldingRequest(
            instrument_uid=Luid,
            tax_lots=[
                models.TargetTaxLotRequest(
                units=index_units,
                cost=models.CurrencyAndAmount(
                    amount=index_cost,
                    currency='GBP'),
                portfolio_cost=index_cost,
                price=index_price)
            ]
        )
    )
# Call LUSID to set all of our holdings to the initial index level
adjust_holdings_response = client.set_holdings(
                              scope=analyst_scope_code,
                              code=index_portfolio_code,
                              effective_at=portfolio_creation_date,
                              holding_adjustments=index_setup)

We now have an index portfolio that we can value.

![Scopes](img/paper-indexportfoliofilled.gif)

In [21]:
# Create our aggregation request
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=analyst_scope_code,
        code='default'),
        effective_at=today,
        metrics=[
            models.AggregateSpec(
                key='Holding/default/PV',
                op='sum'),
            models.AggregateSpec(
                key='Holding/default/Cost',
                op='sum')
        ],
group_by=['Portfolio/default/Name'])
                                                        
# Call LUSID to aggregate across all of our portfolios
aggregated_portfolio = client.get_aggregation_by_portfolio(
    scope=analyst_scope_code,
    code=index_portfolio_code,
    request=aggregation_request
)

# Pretty print the response from LUSID
prettyprint.aggregation_response_index(aggregated_portfolio)

Initial Index Level :1000.0
Current Index Level :995.97
Return :-0.4033%
